In [1]:
import pandas as pd
import re

In [2]:
gdpurl = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita"
gdptables = pd.read_html(gdpurl)
gdptable = gdptables[1]
gdptable.columns = gdptable.columns.droplevel(0) #remove second-level column index
gdptable = gdptable.iloc[:,[0, 6, 1]].sort_values('Estimate', ascending = False) #extract valuable columns and sort descending
gdptable['Country/Territory'] = [next.replace('[n 1]', '').strip('*').upper() for next in \
    [country.encode('ascii', 'ignore').decode('unicode_escape') for country in gdptable['Country/Territory']]] #remove ascii keys and other characters from country names
gdptable = gdptable.reset_index(drop = True) 
gdptable.loc[55, 'Country/Territory'] = 'US VIRGIN ISLANDS' #hard code discrepant country names for future joining
gdptable.loc[57, 'Country/Territory'] = 'SINT MAARTEN'
gdptable.loc[89, 'Country/Territory'] = 'CURACAO'
gdptable.loc[93, 'Country/Territory'] = 'SAINT MARTIN'
gdptable.loc[184, 'Country/Territory'] = 'SAO TOME AND PRINCIPE'

In [3]:
popurl = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density"
poptables = pd.read_html(popurl)
poptable = poptables[0]
poptable = poptable.iloc[:, [1, 4, 6, 3]] #extract valuable columns
poptable['Country/Territory'] = [country.split('(')[0].strip().upper() for country in poptable['Country or dependency']] #extract country name
parents = [] #define parents list appending country name if not a subsidiary and parent country name if a subsidiary (evident by parentheses following entity name)
for country in poptable['Country or dependency']:
    if bool(re.search('\(', country)):
        parent = country.split('(')[1].strip(')').upper()
    else:
        parent = country.split('(')[0].strip().upper()
    parents.append(parent)
poptable['Parent'] = parents #define parent country columns
poptable = poptable.iloc[:, [4, 5, 1, 2, 3]] #extract and rearrange columns
poptable.loc[30, 'Country/Territory'] = 'CURACAO' #hard code discrepant country names for future joining
poptable.loc[60, 'Country/Territory'] = 'SAO TOME AND PRINCIPE'

C:\Users\davis\AppData\Local\Temp\ipykernel_38584\834431728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poptable['Country/Territory'] = [country.split('(')[0].strip().upper() for country in poptable['Country or dependency']] #extract country name
C:\Users\davis\AppData\Local\Temp\ipykernel_38584\834431728.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poptable['Parent'] = parents #define parent country columns


In [4]:
currurl = "https://www.iban.com/currency-codes"
currtables = pd.read_html(currurl)
currtable = currtables[0]
currtable['Country'] = [country.split('(')[0].strip() for country in currtable['Country']] #fix country names
currtable = currtable.iloc[:, [0, 2]] #extract country names and currency codes
currtable.loc[253, 'Country'] = 'UNITED STATES' #hard code discrepant country names for future joining
currtable.loc[35, 'Country'] = 'BRUNEI'
currtable.loc[251, 'Country'] = 'UNITED KINGDOM'
currtable.loc[127, 'Country'] = 'SOUTH KOREA'
currtable.loc[263, 'Country'] = 'US VIRGIN ISLANDS'
currtable.loc[262, 'Country'] = 'BRITISH VIRGIN ISLANDS'
currtable.loc[196, 'Country'] = 'RUSSIA'
currtable.loc[62, 'Country'] = 'CURACAO'
currtable.loc[141, 'Country'] = 'NORTH MACEDONIA'
currtable.loc[261, 'Country'] = 'VIETNAM'
currtable.loc[228, 'Country'] = 'ESWATINI'
currtable.loc[130, 'Country'] = 'LAOS'
currtable.loc[39, 'Country'] = 'CAPE VERDE'
currtable.loc[58, 'Country'] = 'IVORY COAST'
currtable.loc[238, 'Country'] = 'EAST TIMOR'
currtable.loc[233, 'Country'] = 'SYRIA'
currtable.loc[236, 'Country'] = 'TANZANIA'
currtable.loc[126, 'Country'] = 'NORTH KOREA'
currtable.loc[54, 'Country'] = 'DR CONGO'
currtable.loc[len(currtable.index)] = ['MACAU', 'MOP'] #hard code country/territories not found in iban table
currtable.loc[len(currtable.index)] = ['KOSOVO', 'EUR']
currtable.loc[len(currtable.index)] = ['PALESTINE', 'ILS']


C:\Users\davis\AppData\Local\Temp\ipykernel_38584\3384012407.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currtable.loc[253, 'Country'] = 'UNITED STATES' #hard code discrepant country names for future joining
C:\Users\davis\AppData\Local\Temp\ipykernel_38584\3384012407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currtable.loc[35, 'Country'] = 'BRUNEI'
C:\Users\davis\AppData\Local\Temp\ipykernel_38584\3384012407.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [5]:
inter = gdptable.merge(poptable, how = 'inner', left_on = 'Country/Territory', right_on = 'Country/Territory') #join gdp and pop
inter = inter[inter['Country/Territory'].duplicated() == False] #remove repeat countries
inter = inter.merge(currtable, how = 'left', left_on = 'Country/Territory', right_on = 'Country') #join gdp, pop, and curr

In [6]:
expurl = "https://www.cia.gov/the-world-factbook/field/exports/country-comparison/"
exptables = pd.read_html(expurl)
exptable = exptables[0]
exptable['Country'] = [country.upper() for country in exptable['Country']] #fix country names to uppercase for joining
exptable['Exports'] = [int(country.strip('$').replace(',', '')) for country in exptable['Unnamed: 2']] #extract exports as integer
exptable = exptable.iloc[:, [1, 4]] #extract country name and exports

In [7]:
impurl = "https://www.cia.gov/the-world-factbook/field/imports/country-comparison/"
imptables = pd.read_html(impurl)
imptable = imptables[0]
imptable['Country'] = [country.upper() for country in imptable['Country']] #fix country names to uppercase for joining
imptable['Imports'] = [int(country.strip('$').replace(',', '')) for country in imptable['Unnamed: 2']] #extract imports as integer
imptable = imptable.iloc[:, [1, 4]] #extract country name and exports

In [8]:
trade = exptable.merge(imptable, how = 'outer', left_on = 'Country', right_on = 'Country') #join exports and imports, hardcoding Liechtenstein below using data from same source just one year earlier
trade.loc[142, 'Exports'] = 3774000000 #https://www.cia.gov/the-world-factbook/countries/liechtenstein/#economy
trade.loc[142, 'Imports'] = 2230000000 #https://www.cia.gov/the-world-factbook/countries/liechtenstein/#economy
trade['Ratio'] = trade['Exports'] / trade['Imports'] #define trade ratio columns
trade.loc[197, 'Country'] = 'FALKLAND ISLANDS' #hard code country names
trade.loc[7, 'Country'] = 'SOUTH KOREA'
trade.loc[33, 'Country'] = 'CZECH REPUBLIC'
trade.loc[162, 'Country'] = 'US VIRGIN ISLANDS'
trade.loc[28, 'Country'] = 'TURKEY'
trade.loc[143, 'Country'] = 'BAHAMAS'
trade.loc[215, 'Country'] = 'SAINT HELENA, ASCENSION AND TRISTAN DA CUNHA'
trade.loc[188, 'Country'] = 'CAPE VERDE'
trade.loc[93, 'Country'] = 'IVORY COAST'
trade.loc[153, 'Country'] = 'EAST TIMOR'
trade.loc[85, 'Country'] = 'MYANMAR'
trade.loc[202, 'Country'] = 'MICRONESIA'
trade.loc[130, 'Country'] = 'CONGO'
trade.loc[205, 'Country'] = 'GAMBIA'
trade.loc[199, 'Country'] = 'NORTH KOREA'
trade.loc[82, 'Country'] = 'DR CONGO' #hard code exports/imports for missing countries
trade.loc[len(trade.index)] = ['ISLE OF MAN', 432000000, 922000000, 432000000/922000000] #https://assets.publishing.service.gov.uk/media/653fbb156de3b9000da7a609/isle-of-man-trade-and-investment-factsheet-2023-11-01.pdf
trade.loc[len(trade.index)] = ['JERSEY', 3900000000, 4300000000, 3900000000/4300000000] #https://assets.publishing.service.gov.uk/media/653fbe9246532b000d67f545/jersey-trade-and-investment-factsheet-2023-11-01.pdf
trade.loc[len(trade.index)] = ['GUERNSEY', 953000000, 3100000000, 953000000/3100000000] #https://assets.publishing.service.gov.uk/media/653fba4746532b001467f52f/guernsey-trade-and-investment-factsheet-2023-11-01.pdf
trade.loc[len(trade.index)] = ['SAINT MARTIN', 23700000, 529000000, 23700000/529000000] #https://oec.world/en/profile/country/maf
trade.loc[len(trade.index)] = ['PALESTINE', 1450000000, 6940000000, 1450000000/6940000000] #https://oec.world/en/profile/country/pse

In [9]:
final = inter.merge(trade, how = 'left', left_on = 'Country/Territory', right_on = 'Country') #merge all
final = final.iloc[:, [0, 3, 2, 8, 1, 4, 5, 6, 12]] #rearrange columns
final.columns = ['Entity', 'Parent', 'Region', 'Currency', 'GDP', 'Population', 'Area', 'Density', 'Ratio'] #rename columns
final.iloc[final[final['Parent'] == 'MICRONESIA'].index, 0] = 'FEDERATED STATES OF MICRONESIA' #fix parent country names to correct/consistent names
final.iloc[final[final['Parent'] == 'CONGO'].index, 0] = 'REPUBLIC OF THE CONGO'
final.iloc[final[final['Parent'] == 'DR CONGO'].index, 0] = 'DEMOCRATIC REPUBLIC OF THE CONGO'
final.iloc[final[final['Parent'] == 'UK'].index, 1] = 'UNITED KINGDOM'
final.iloc[final[final['Parent'] == 'US'].index, 1] = 'UNITED STATES'
final.iloc[final[final['Parent'] == 'NL'].index, 1] = 'NETHERLANDS'
final.iloc[final[final['Parent'] == 'NZ'].index, 1] = 'NEW ZEALAND'
final.iloc[final[final['Parent'] == 'COOK ISLANDS'].index, 1] = 'NEW ZEALAND'
final.iloc[final[final['Parent'] == 'NIUE'].index, 1] = 'NEW ZEALAND'
final.iloc[final[final['Parent'] == 'SÃO TOMÉ AND PRÍNCIPE'].index, 1] = 'SAO TOME AND PRINCIPE'
final.iloc[final[final['Parent'] == 'CONGO'].index, 1] = 'REPUBLIC OF THE CONGO'
final.iloc[final[final['Parent'] == 'DR CONGO'].index, 1] = 'DEMOCRATIC REPUBLIC OF THE CONGO'
final.iloc[final[final['Parent'] == 'METROPOLITAN'].index, 1] = 'FRANCE'
final.iloc[final[final['Parent'] == 'MICRONESIA'].index, 1] = 'FEDERATED STATES OF MICRONESIA'
final.iloc[final[final['Parent'] == 'EXCLUDING ANTARCTICA'].index, 1] = 'WORLD'
final.drop(index = [8, 10, 75, 81, 86, 100, 117, 136, 147, 155, 158, 166, 170, 210, 219], inplace = True) #remove country indexes duplicated because of currencies (most official currency favored)
final = final.reset_index(drop = True)

In [10]:
final.to_csv('gdp.csv', index = None) #send to csv